# Analyze probing results
- Preprocess a bit. Select the best results from each configuration. Export to the same directory.

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [6]:
def analyze_results(report_dir):
    dfs = []
    for p in Path(report_dir).iterdir():
        if not p.name.startswith("report"): 
            continue
        report_df = pd.read_csv(p)
        print ("   ", p.name, report_df.shape)
        dfs.append(report_df)
    df = pd.concat(dfs, axis=0)
    
    tasks = []
    layers = []
    for i, row in df.iterrows():
        tasks.append("_".join(row["task"].split("_")[:-2]))
        layers.append(int(row["task"].split("_")[-1]))
    df.drop(columns=["task"], inplace=True)
    df["task"] = tasks
    df["layer"] = layers
    return df 


all_model_best_results = []
for p in Path("../reports").iterdir():
    
    if not p.is_dir():
        continue
    print(p.name)
    df_best_results = analyze_results(p)
    df_best_results["LM"] = [p.name] * len(df_best_results)
    all_model_best_results.append(df_best_results) 
df_all = pd.concat(all_model_best_results, axis=0)
# There are 25 LM x 13 layers x 7 (probing) tasks x 3 configs x 7 models x 5 rs
# For each {LM, layer, task, config}, average by rs, and take the best model.
df_avg = df_all.groupby(["LM", "layer", "task", "config", "model"], as_index=False).mean()
df_best_ids = df_avg.groupby(["LM", "layer", "task", "config"], as_index=False)["val_acc"].idxmax()["val_acc"]
df_best = df_avg.iloc[df_best_ids]

embeddings_albert_base_v2_corr_500
    report_subj_number.csv (1365, 12)
    report_tree_depth.csv (1365, 12)
    report_obj_number.csv (1365, 12)
    report_coordination_inversion.csv (1365, 12)
    report_odd_man_out.csv (1365, 12)
    report_past_present.csv (1365, 12)
    report_bigram_shift.csv (1365, 12)
embeddings_roberta_base_corr_1000
    report_subj_number.csv (1365, 12)
    report_tree_depth.csv (1365, 12)
    report_obj_number.csv (1365, 12)
    report_coordination_inversion.csv (1365, 12)
    report_odd_man_out.csv (1365, 12)
    report_past_present.csv (1365, 12)
    report_bigram_shift.csv (1365, 12)
embeddings_xlm_roberta_base_corr_4000
    report_subj_number.csv (1365, 12)
    report_tree_depth.csv (1365, 12)
    report_obj_number.csv (1365, 12)
    report_coordination_inversion.csv (1365, 12)
    report_odd_man_out.csv (1365, 12)
    report_past_present.csv (1365, 12)
    report_bigram_shift.csv (1365, 12)
embeddings_roberta_base_corr_6000
    report_subj_number.csv (

In [7]:
df_best.shape

(6825, 14)

In [8]:
df_best.to_csv("../reports/probing_1200_per_class.csv", index=False)